In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [14]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode
import time

Conectando ao Neo4j 

In [15]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [16]:
# Query to get theses and their repositories from the Neo4j database
# This query retrieves theses that are published by the Federal University of Santa Catarina (UFSC) and have not been processed yet.
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

# Alternative query to get theses that do not have excerpts and have been processed.
query2 = """ 
match (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
where (not (t:Thesis)-[:BFO_0000050]-(:Exerpt)) and  exists {(t{processed:true})}
return distinct(t["uri"]) as Thesis, t["repository"] as repository
"""

# Query to retrive theses with few pages and their repositories.
query3 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
Match (t:Thesis)-[:BFO_0000050]-(e:Exerpt)
RETURN count(e) as pages, t["uri"] as Thesis, t["repository"] as repository
 order by (pages) limit 40
"""

query4 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["title"] as title 
"""

query5 = """
MATCH (t:Thesis {processed: true})-[:publisher]-(uni:University {uri: 'tag:stardog:api:UFSC'})
WHERE NOT (t)--(:Exerpt)
RETURN distinct(t["uri"]) as Thesis, t["title"] as title 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query4).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
#thesis_repository = pd.DataFrame(results, columns=["pages", "Thesis", "repository"])
thesis_repository = pd.DataFrame(results, columns=["Thesis", "title"])
thesis_repository = thesis_repository[9:].reset_index(drop=True)  #[7:]
thesis_repository

,Thesis,title
0,tag:stardog:api:entre_o_diario_e_o_anuario_a_q...,Entre o diário e o anuário: a queda ecoa no ap...
1,tag:stardog:api:por_historias_indigenas_o_povo...,Por histórias indígenas: o povo Makurap e o oc...
2,tag:stardog:api:o_que_e_ser_um_crackudo_proces...,O que é ser um crackudo?: processos de desuman...
3,tag:stardog:api:decolonialidade_quadrinistica_...,Decolonialidade quadrinística na educação em c...
4,tag:stardog:api:educacao_das_relacoes_etnicora...,Educação das relações étnico-raciais: branquit...
...,...,...
719,tag:stardog:api:jequitiboaba_do_samba_o_eco_da...,Jequitiboabá do samba: o eco da voz do dono
720,tag:stardog:api:avaliacao_da_evapotranspiracao...,Avaliação da evapotranspiração real da bacia d...
721,tag:stardog:api:a_protecao_integral_de_crianca...,A Proteção integral de crianças e adolescentes...
722,tag:stardog:api:ue_gaucho_em_floripa_tem_samba...,"Ué gaúcho, em Floripa tem samba?: uma antropol..."


Criando as utils functions

In [17]:
# Funcao para coletar link do repositorio na página da BDTD

def get_repository_link(title):

    bdtd_url = 'https://bdtd.ibict.br/vufind/Search/Results?lookfor=' + title.replace(' ', '+') + '&type=Title' #&limit=1'
    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(bdtd_url, verify=True).text#, verify=False) 
    except:
        print("Sleeping for 5 seconds before retrying... (get_repository_link)")
        time.sleep(5)
        try:
            f = requests.get(bdtd_url, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying... (get_repository_link)")
            time.sleep(30)
            f = requests.get(bdtd_url, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")

    #Coletando link para o repositório
    link_list = []
    for doc in soup.find_all('a', href=True):

        if (doc['href'].startswith('https://repositorio.ufsc.br/') |
            doc['href'].startswith('http://repositorio.ufsc.br/')):
            path = doc['href']
            link_list.append(path)
    if link_list != []:
        return list(set(link_list))[0]
    else:
        return None



# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(url, verify=True).text#, verify=False) 
    except:
        print("Sleeping for 5 seconds before retrying... (get_pdf_link)")
        time.sleep(5)
        try:
            f = requests.get(url, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying... (get_pdf_link)")
            time.sleep(30)
            f = requests.get(url, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    link_list = []
    for doc in soup.find_all('a', href=True):
        if doc['href'].startswith('/bitstream'):     # (doc['href'].endswith('.pdf')):# & doc['href'].startswith('/bitstream')):
            path = doc['href']
            prefix_uni = 'https://repositorio.ufsc.br'
            link = prefix_uni + path
            link_list.append(link)
    if link_list != []:
        return list(set(link_list))
    else:
        return None
    
    #return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):


    # NOTE the stream=True parameter below
    try:
        r = requests.get(pdf_link, verify=True, stream=True)
    except:
        print("Sleeping for 5 seconds before retrying... (download_file)")
        time.sleep(5)
        try:
            r = requests.get(pdf_link, verify=True, stream=True)
        except:
            print("Sleeping for 30 seconds before retrying... (download_file)")
            time.sleep(30)
            r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese UFSC.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    try:
        reader = PdfReader(tese_path)
        number_of_pages = len(reader.pages)
    except:
        print(f"Erro ao ler o arquivo {tese_path}. ")
        excerpt = {}
        return excerpt
    
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        
        # Extraindo o texto da página
        try:
            if page_number == 0:
                try:
                    page_text = (reader.pages[page_number].extract_text() + '\n ' 
                                + reader.pages[page_number+1].extract_text()[:400])
                except:
                    page_text = reader.pages[page_number].extract_text()
            else:
                
                if page_number == number_of_pages -1:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text())
                    except:
                        page_text = reader.pages[page_number].extract_text()
                
                else:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text() + '\n ' 
                                    + reader.pages[page_number+1].extract_text()[:400])  
                    except:
                        page_text = reader.pages[page_number].extract_text()
        except:
            page_text = ''
        
        # Limpando page text         
        page_text = page_text.replace("'", "").replace('"', '')
        # Verifica se o último caractere é uma barra invertida
        if page_text != '':
            if page_text[-1] == '\\': 
                page_text = page_text[:-2]
        page_text = re.sub(r"[\ud800-\udfff]", " ", page_text)
        page_text = page_text.encode('utf-8').decode('utf-8')
        
        
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        

        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [ ]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    title = thesis_repository['title'][i].replace("?","").replace('-', ' ').replace('&', ' ')  
    repo = get_repository_link(title)

    # Coletando o link do PDF
    #url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(repo)
    if pdf_link is not None:
        # Itereando pelos links de PDF encontrados (em alguns casos há outros materiais além da tese)
        for pdf_num in range(len(pdf_link)):
            print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link[pdf_num]}")
            download_file(pdf_link[pdf_num], thesis_uri)

            # Extraindo o texto e entidades
            tese_path = 'tese UFSC.pdf'
            excerpt = extratc_text(tese_path)

            excerpts = []
            persons = []
            gpes = []

            # Salvando as triplas em CSV
            for n in excerpt.keys():

                excerpt_uri = thesis_repository['Thesis'][i] + '_pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_text = excerpt[n]['text']
                # Verifica se o último caractere é uma barra invertida
                #if excerpt_text[-1] == '\\': 
                #    excerpt_text = excerpt_text[:-1]
                

                excerpt_page = 'pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_lang = excerpt[n]['lang']
                
                excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, excerpt_page, excerpt_lang])
                    
                #Criando as relações com as entidades
                for person in excerpt[n]['persons']:
                    persons.append([excerpt_uri, 'tag:stardog:api:' + person])
                
                for gpe in excerpt[n]['gpes']:
                    gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
            
            # Salvando os dados em CSV no diretório de importação do Neo4j
            pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFSC.csv', index=False, header=False)
            pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFSC.csv', index=False, header=False)
            pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFSC.csv', index=False, header=False)

            #Deletando o arquivo PDF após o processamento
            os.remove('tese UFSC.pdf')

            print(f"Loading triplas: {thesis_uri}")

            # Importanto os dados para o Neo4j
            #Excerpts
            query = """
            LOAD CSV FROM 'file:///excerptsUFSC.csv'
            AS row
            MATCH  (t:Thesis{uri: row[0]})
            MERGE  (e:Exerpt{
                    uri: row[1],
                    text: row[2],
                    page: row[3],
                    lang: row[4]
                    })-[r:BFO_0000050]-(t);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #Persons
            query = """
            LOAD CSV FROM 'file:///personsUFSC.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Person{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #GPEs
            query = """
            LOAD CSV FROM 'file:///gpesUFSC.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Place{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            # Removendo os arquivos CSV após a importação
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFSC.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFSC.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFSC.csv')

            print(f"Total time: {pd.Timestamp.now() - start_time}")
            print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
            # Atualizando a hora parcial
            parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    SET t.repository = \'""" + str(repo) +  """\'
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()


0 - Downloading and extracting: entre_o_diario_e_o_anuario_a_queda_ecoa_no_apocalipse from https://repositorio.ufsc.br/bitstream/handle/123456789/229255/PLIT0889-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Erro ao ler o arquivo tese UFSC.pdf. 
Loading triplas: entre_o_diario_e_o_anuario_a_queda_ecoa_no_apocalipse
Total time: 0 days 00:00:21.116527
Partial time: 0 days 00:00:21.116527
1 - Downloading and extracting: por_historias_indigenas_o_povo_makurap_e_o_ocupar_seringalista_na_amazonia from https://repositorio.ufsc.br/bitstream/handle/123456789/226949/PHST0719-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Erro ao ler o arquivo tese UFSC.pdf. 
Loading triplas: por_historias_indigenas_o_povo_makurap_e_o_ocupar_seringalista_na_amazonia
Total time: 0 days 00:00:41.959607
Partial time: 0 days 00:00:20.843080
2 - Downloading and extracting: o_que_e_ser_um_crackudo_processos_de_desumanizacao_e_ceticismo_misantropico_na_contemporaneidade from https://repositorio.ufsc.br/bitstream/handle/123456789/230968/PICH0235-T.pdf?sequence=-1&isAllowed=y


...

Erro ao ler o arquivo tese UFSC.pdf. 
Loading triplas: sob_suspeita_relacoes_politicas_e_corrupcao_no_servico_de_protecao_aos_indios_ao_raiar_do_regime_militar_19631967
Total time: 0 days 00:36:08.278660
Partial time: 0 days 00:00:16.354950
58 - Downloading and extracting: synthesis_and_processing_of_ti3sic2based_inks_via_inkjet_and_screen_printing_technology_for_eletronic_applications from https://repositorio.ufsc.br/bitstream/handle/123456789/189917/PCEM0467-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Erro ao ler o arquivo tese UFSC.pdf. 
Loading triplas: synthesis_and_processing_of_ti3sic2based_inks_via_inkjet_and_screen_printing_technology_for_eletronic_applications
Total time: 0 days 00:36:26.095584
Partial time: 0 days 00:00:17.816924
59 - Downloading and extracting: trabalho_decente_e_bem_viver_enlace_necessario_para_a_construcao_do_vinculo_comunitario_fundamentado_no_reconhecimento_relacionalidade_e_reciprocidade from https://repositorio.ufsc.br/bitstream/handle/123456789/215411/PDPC1474-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(3)


Loading triplas: trabalho_decente_e_bem_viver_enlace_necessario_para_a_construcao_do_vinculo_comunitario_fundamentado_no_reconhecimento_relacionalidade_e_reciprocidade
Total time: 0 days 00:37:30.937574
Partial time: 0 days 00:01:04.841990
No PDF link found for ocupar_a_universidade_mas_manter_o_coracao_na_comunidade_caminhos_de_estudantes_laklanoxokleng_na_ufsc
